In [2]:
import cv2 as cv
import numpy as np 
from matplotlib import pyplot as plt


# Pipeline for object tracking 

1) Image Acquisition --capture and read frame from the video
2) Image preparation -- Gaussian soothing takes away higher frequency value and edges 
3) color Recognition -- BGR->HSV and thresold based on marker color 
4) Object Identification -- contours for colored object , with max area ,features of contours with max area




In [8]:
img =  cv.imread('pic1.png')

img_gray =  cv.cvtColor(img,cv.COLOR_BGR2GRAY)
ret,thresh = cv.threshold(img_gray,127,255,0)  # 0: binary type 

# IMAGE , MODE : CONTOUR RETRIVAL MODE RETT_TREE : retives all the contours and reconstructs a full hierarchy of nested contours 
#METHOD : CONTOUR APPROXIMATION METHOD  HOW  one contour is related to another , parent child relation 

# CHAIN_APPROX_NONE : stores absolutely all contour points 
contours,hierachy = cv.findContours(thresh,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)

#cv.drawContours(img,contours,-1,(0,255,0),3)
# 3rd : contour index indicating parameters to be drawn , -1 all contors are drawn 
#4th : color of the contour to be drawn 

# to access the 10th contour 
cnt =contours[10]
cv.drawContours(img,[cnt],0,(0,255,0),3)
cv.imshow('contours',img)
cv.waitKey(0)
cv.destroyAllWindows()

In [16]:

img =  cv.imread('pic1.png')

img_gray =  cv.cvtColor(img,cv.COLOR_BGR2GRAY)

ret,thresh = cv.threshold(img_gray,127,255,0)  # 0: binary type 

contours,hierachy = cv.findContours(thresh,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)

# to access the 6th contour 
cnt =contours[6]
cv.drawContours(img,[cnt],0,(0,255,0),3)

# to calculate the center of mass of the facial contour 
# takes input array of contours ,this returns all the moments calculated for the img in dict format
M= cv.moments(cnt)
#using this we can extract features like center of mass , area
cx = int(M['m10']/M['m00'])
cy = int(M['m01']/M['m00'])

print('centroid of the face is ',(cx,cy))

area = cv.contourArea(cnt)
print('area of the face is ',area)

perimeter = cv.arcLength(cnt,True)
print('perimeter of the face is ',perimeter)

x,y,w,h = cv.boundingRect(cnt)
#x,y -- top left coordinates of the rectangle 
# w -width of the rectangle 
# h -height of the rect 

cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)


(x,y),radius = cv.minEnclosingCircle(cnt)
center = (int(x),int(y))
radius = int(radius)
cv.circle(img,center, radius,(255,0,0),2)


cv.imshow('contours',img)
cv.waitKey(0)
cv.destroyAllWindows()

centroid of the face is  (55, 218)
area of the face is  8785.0
perimeter of the face is  388.7350620031357


In [24]:
vid = cv.VideoCapture('vtest.avi')

lower_yellow = np.array([0,125,125])
upper_yellow = np.array([10,255,255])

while(vid.isOpened()):
    
    #capture frame by frame 
    ret,frame = vid.read()
    
    
    image_smooth = cv.GaussianBlur(frame,(7,7),0)
    
    # threshold the image for the yellow
    image_hsv = cv.cvtColor(frame,cv.COLOR_BGR2HSV)
    image_threshold = cv.inRange(image_hsv,lower_yellow,upper_yellow)
    # 1 for with in the bound else 0
    
    # find contour 
    contours,hierachy = cv.findContours(image_threshold,cv.RETR_TREE,cv.CHAIN_APPROX_NONE)
    
    #find the index of the largest contour
    if(len(contours)!=0):
        areas = [cv.contourArea(c) for c in contours]
        max_index = np.argmax(areas)
        cnt = contours[max_index]
        x,y,w,h = cv.boundingRect(cnt)
        cv.rectangle(frame ,(x,y),(x+w,y+h),(255,0,0),2)
    # Display the resulting frame 
    cv.imshow('frame',frame)
    
    key = cv.waitKey(100)
    if key ==27:
        break

vid.release()
cv.destroyAllWindows()

In [29]:
vid = cv.VideoCapture(0)

lower_yellow = np.array([0,125,125])
upper_yellow = np.array([10,255,255])

while(vid.isOpened()):
    
    #capture frame by frame 
    ret,frame = vid.read()
    frame = cv.flip(frame,1) 
    # 2nd : determines the axis around which to be flipped 0: x axis , 1 : y axis , -1 both axis 
    
    w = frame.shape[1]
    h = frame.shape[0]
    
    image_smooth = cv.GaussianBlur(frame,(7,7),0)
    
    # defining the region of interest
    mask = np.zeros_like(frame)
    mask[50:350 , 50:350] = [255,255,255]
    image_roi = cv.bitwise_and(image_smooth,mask)
    cv.rectangle(frame ,(50,50),(350,350),(0,0,255),2)
    cv.line(frame ,(150,50),(150,350),(0,0,255),1)
    cv.line(frame ,(250,50),(250,350),(0,0,255),1)
    cv.line(frame ,(50,150),(350,150),(0,0,255),1)
    cv.line(frame ,(50,250),(350,250),(0,0,255),1)
    
    
    # threshold the image for the yellow
    image_hsv = cv.cvtColor(image_roi,cv.COLOR_BGR2HSV)
    image_threshold = cv.inRange(image_hsv,lower_yellow,upper_yellow)
    # 1 for with in the bound else 0
    
    # find contour 
    contours,hierachy = cv.findContours(image_threshold,cv.RETR_TREE,cv.CHAIN_APPROX_NONE)
    
    #find the index of the largest contour
    if(len(contours)!=0):
        areas = [cv.contourArea(c) for c in contours]
        max_index = np.argmax(areas)
        cnt = contours[max_index]
        
        
        #pointer on video
        M= cv.moments(cnt)
        if(M['m00']!=0):
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            cv.circle(frame,(cx,cy),10,(0,255,0),-1)
            
            #cursor motion
            if cx in range(150,250):
                if cy <150:
                    print("upper middle")
                    
                elif cy > 250:
                    print ("lower middle")
                    
                else:
                    print("center")
            if cy in range(150,250):
                if cx <150:
                    print("left middle")
                    
                elif cx > 250:
                    print ("right middle")
                    
                else:
                    print("center")
        
    # Display the resulting frame 
    cv.imshow('frame',frame)
    
    key = cv.waitKey(100)
    if key ==27:
        break

vid.release()
cv.destroyAllWindows()

lower middle
lower middle
lower middle
lower middle
lower middle
lower middle
lower middle
lower middle
lower middle
lower middle
lower middle
lower middle
right middle
right middle
right middle
right middle
right middle
right middle
right middle
right middle
center
center
left middle
lower middle
lower middle
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
center
left middle
center
center
right middle
left middle
right middle
right middle
right middle
right middle
right middle
right middle
lower middle
left middle
upper middle
right middle
right middle
left middle
left middle
left middle
left middle
left middle
left middle
left middle
left middle
left middle
left middle
left middle
left middle
left middle
left middle
left middle
left middle
left middle
upper middle
right middle